In [1]:
import os
import time
import glob
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.manifold import TSNE
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# ------------------------------------------------------------------------------
# 1. MHEALTH Dataset
# ------------------------------------------------------------------------------
def _load_single_mhealth_log(path: str, feature_cols: list[str]):
    df = pd.read_csv(
        path,
        sep="\t",
        header=None,
        names=feature_cols + ["label"],
    )
    return df

def load_mhealth_dataframe(data_dir: str):
    feature_cols = [
        "acc_chest_x", "acc_chest_y", "acc_chest_z",      # 0-2
        "ecg_1", "ecg_2",                                 # 3-4
        "acc_ankle_x", "acc_ankle_y", "acc_ankle_z",      # 5-7
        "gyro_ankle_x", "gyro_ankle_y", "gyro_ankle_z",   # 8-10
        "mag_ankle_x", "mag_ankle_y", "mag_ankle_z",      # 11-13
        "acc_arm_x", "acc_arm_y", "acc_arm_z",            # 14-16
        "gyro_arm_x", "gyro_arm_y", "gyro_arm_z",         # 17-19
        "mag_arm_x", "mag_arm_y", "mag_arm_z",            # 20-22
    ]

    log_files = glob.glob(os.path.join(data_dir, "mHealth_subject*.log"))
    if not log_files:
         raise FileNotFoundError(f"No mHealth_subject*.log files found in {data_dir}")
    print(f"Found {len(log_files)} log files in {data_dir}")

    dfs = []
    for fp in log_files:
        df_i = _load_single_mhealth_log(fp, feature_cols)
        dfs.append(df_i)

    full_df = pd.concat(dfs, ignore_index=True)
    full_df = full_df[full_df["label"] != 0].copy()
    full_df.loc[:, "label"] = full_df["label"] - 1

    return full_df, feature_cols

def create_mhealth_windows(df: pd.DataFrame, feature_cols: list[str], window_size: int, step_size: int):
    data_arr = df[feature_cols].to_numpy(dtype=np.float32)
    labels_arr = df["label"].to_numpy(dtype=np.int64)
    L = data_arr.shape[0]

    X_list = []
    y_list = []

    start = 0
    while start + window_size <= L:
        end = start + window_size
        window_x = data_arr[start:end]
        window_label = labels_arr[end - 1]
        window_x_ct = np.transpose(window_x, (1, 0))

        X_list.append(window_x_ct)
        y_list.append(int(window_label))
        start += step_size

    if not X_list:
        raise RuntimeError("No windows created.")

    X_np = np.stack(X_list, axis=0).astype(np.float32)
    y_np = np.array(y_list, dtype=np.int64)
    return X_np, y_np

class MHEALTHDataset(Dataset):
    def __init__(self, data_dir: str, window_size: int = 128, step_size: int = 64):
        super().__init__()
        full_df, feature_cols = load_mhealth_dataframe(data_dir)
        X, y = create_mhealth_windows(full_df, feature_cols, window_size, step_size)

        self.X = np.transpose(X, (0, 2, 1)).astype(np.float32)
        self.y = y

        self.label_names = [
            "Standing still", "Sitting", "Lying down",   # 0, 1, 2 (Static)
            "Walking", "Climbing stairs", "Waist bends", # 3, 4, 5
            "Arms elevation", "Knees bending", "Cycling",# 6, 7, 8
            "Jogging", "Running", "Jump front&back",     # 9, 10, 11
        ]

        print("=" * 80)
        print("Loaded MHEALTH dataset")
        print(f"  X shape : {self.X.shape}  (N, T, C)")
        print(f"  y shape : {self.y.shape}  (N,)")
        print(f"  Classes : {len(self.label_names)}")
        print("=" * 80)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx: int):
        # subject 정보가 필요하다면 3번째 반환값으로 넣고 collate_fn을 수정해야 함.
        return torch.FloatTensor(self.X[idx]), torch.LongTensor([self.y[idx]])[0]

# ------------------------------------------------------------------------------
# 2. ASF Model Components
# ------------------------------------------------------------------------------

class LatentEncoder(nn.Module):
    def __init__(self, input_channels=9, latent_dim=64):
        super().__init__()
        self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(64, latent_dim, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(latent_dim)

    def forward(self, x):
        x = x.transpose(1, 2)
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        s = F.relu(self.bn3(self.conv3(h)))
        s = s.transpose(1, 2)
        return s

class FlowComputer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, s):
        B, T, D = s.shape

        flow_raw = s[:, 1:, :] - s[:, :-1, :]
        flow_mag = torch.norm(flow_raw, dim=-1, keepdim=True)
        flow_dir = flow_raw / (flow_mag + 1e-8)

        flow_features = torch.cat(
            [flow_raw, flow_mag.expand(-1, -1, D), flow_dir],
            dim=-1
        )
        return flow_features, flow_raw, flow_mag

class FlowEncoder(nn.Module):
    def __init__(self, flow_dim, hidden_dim=64, num_heads=4):
        super().__init__()
        self.flow_embed = nn.Linear(flow_dim, hidden_dim)
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads,
            batch_first=True
        )
        self.flow_conv1 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.flow_conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=1, padding=0)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

    def forward(self, flow_features):
        h = self.flow_embed(flow_features)
        h_att, _ = self.attention(h, h, h)

        h_att = h_att.transpose(1, 2)
        h = F.relu(self.bn1(self.flow_conv1(h_att)))
        h = F.relu(self.bn2(self.flow_conv2(h)))

        h_pool = torch.mean(h, dim=-1)
        return h_pool

class StateTransitionPredictor(nn.Module):
    def __init__(self, latent_dim=64, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, s_t):
        B, Tm1, D = s_t.shape
        inp = s_t.reshape(B * Tm1, D)
        out = self.net(inp)
        return out.reshape(B, Tm1, D)

class ASFDCLClassifier(nn.Module):
    def __init__(self,
                 input_channels=9,
                 latent_dim=64,
                 hidden_dim=64,
                 num_classes=6,
                 num_heads=4,
                 projection_dim=128):
        super().__init__()

        self.num_classes = num_classes
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim

        self.latent_encoder = LatentEncoder(input_channels, latent_dim)
        self.flow_computer = FlowComputer()
        self.flow_encoder = FlowEncoder(latent_dim * 3, hidden_dim, num_heads)
        self.state_predictor = StateTransitionPredictor(latent_dim, hidden_dim)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

        self.flow_prototypes = nn.Parameter(
            torch.randn(num_classes, hidden_dim)
        )

        self.projection_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, projection_dim)
        )

    def forward(self, x, return_details=False):
        s = self.latent_encoder(x)

        s_t = s[:, :-1, :]
        s_next = s[:, 1:, :]
        s_pred_next = self.state_predictor(s_t)

        flow_features, flow_raw, flow_mag = self.flow_computer(s)

        h = self.flow_encoder(flow_features)

        z = self.projection_head(h)
        z = F.normalize(z, dim=-1)

        logits = self.classifier(h)

        if not return_details:
            return logits

        details = {
            "s": s,
            "s_t": s_t,
            "s_next": s_next,
            "s_pred_next": s_pred_next,
            "flow_features": flow_features,
            "flow_raw": flow_raw,
            "flow_mag": flow_mag,
            "h": h,
            "z": z,
            "prototypes": self.flow_prototypes
        }
        return logits, details


# ------------------------------------------------------------------------------
# 3. Dynamics-aware Contrastive Loss
# ------------------------------------------------------------------------------
def compute_contrastive_loss(z, labels, temperature=0.07):
    B = z.shape[0]
    device = z.device

    sim_matrix = torch.mm(z, z.t()) / temperature

    labels_expanded = labels.unsqueeze(1)
    positive_mask = (labels_expanded == labels_expanded.t()).float()

    positive_mask = positive_mask - torch.eye(B, device=device)

    mask = torch.eye(B, device=device).bool()
    sim_matrix_masked = sim_matrix.masked_fill(mask, float('-inf'))

    exp_sim = torch.exp(sim_matrix_masked)

    pos_sim = (exp_sim * positive_mask).sum(dim=1)

    all_sim = exp_sim.sum(dim=1)

    has_positive = positive_mask.sum(dim=1) > 0

    if has_positive.sum() == 0:
        return torch.tensor(0.0, device=device)

    loss = -torch.log(pos_sim[has_positive] / (all_sim[has_positive] + 1e-8))

    return loss.mean()


# ------------------------------------------------------------------------------
# 4. ASF-DCL Losses: CE + L_dyn + L_flow_prior + L_proto + L_contrast
# ------------------------------------------------------------------------------
def compute_asf_dcl_losses(logits, details, labels,
                           lambda_dyn=0.1,
                           lambda_flow=0.05,
                           lambda_proto=0.1,
                           lambda_contrast=0.15,
                           dyn_classes=(0, 1, 2),
                           static_classes=(3, 4, 5),
                           dyn_target=0.7,
                           static_target=0.1,
                           proto_tau=0.1,
                           contrast_temp=0.07):
    device = logits.device

    cls_loss = F.cross_entropy(logits, labels, label_smoothing=0.05)

    s_next = details["s_next"]
    s_pred_next = details["s_pred_next"]
    dyn_loss = F.mse_loss(s_pred_next, s_next)

    flow_mag = details["flow_mag"]
    B, Tm1, _ = flow_mag.shape
    flow_mean = flow_mag.mean(dim=1).view(B)

    dyn_mask = torch.zeros_like(flow_mean, dtype=torch.bool)
    static_mask = torch.zeros_like(flow_mean, dtype=torch.bool)
    for c in dyn_classes:
        dyn_mask = dyn_mask | (labels == c)
    for c in static_classes:
        static_mask = static_mask | (labels == c)

    flow_prior_loss = torch.tensor(0.0, device=device)
    if dyn_mask.any():
        dyn_flow = flow_mean[dyn_mask]
        flow_prior_loss = flow_prior_loss + F.mse_loss(
            dyn_flow, torch.full_like(dyn_flow, dyn_target)
        )
    if static_mask.any():
        static_flow = flow_mean[static_mask]
        flow_prior_loss = flow_prior_loss + F.mse_loss(
            static_flow, torch.full_like(static_flow, static_target)
        )

    h = details["h"]
    prototypes = details["prototypes"]

    h_norm = F.normalize(h, dim=-1)
    proto_norm = F.normalize(prototypes, dim=-1)

    sim = h_norm @ proto_norm.t()
    proto_logits = sim / proto_tau
    proto_loss = F.cross_entropy(proto_logits, labels, label_smoothing=0.05)

    z = details["z"]
    contrast_loss = compute_contrastive_loss(z, labels, temperature=contrast_temp)

    total_loss = (
        cls_loss +
        lambda_dyn * dyn_loss +
        lambda_flow * flow_prior_loss +
        lambda_proto * proto_loss +
        lambda_contrast * contrast_loss
    )

    loss_dict = {
        "total": total_loss.item(),
        "cls": cls_loss.item(),
        "dyn": dyn_loss.item(),
        "flow_prior": flow_prior_loss.item(),
        "proto": proto_loss.item(),
        "contrast": contrast_loss.item()
    }
    return total_loss, loss_dict


# ------------------------------------------------------------------------------
# 5. Train / Evaluation
# ------------------------------------------------------------------------------
def train_epoch(model, dataloader, optimizer, device,
                lambda_dyn=0.1, lambda_flow=0.05,
                lambda_proto=0.1, lambda_contrast=0.15, **kwargs):
    model.train()
    total_loss = 0

    all_preds = []
    all_labels = []

    loss_accumulator = {
        "cls": 0.0,
        "dyn": 0.0,
        "flow_prior": 0.0,
        "proto": 0.0,
        "contrast": 0.0
    }

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        logits, details = model(x, return_details=True)
        loss, loss_dict = compute_asf_dcl_losses(
            logits, details, y,
            lambda_dyn=lambda_dyn,
            lambda_flow=lambda_flow,
            lambda_proto=lambda_proto,
            lambda_contrast=lambda_contrast,
            **kwargs
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        for k in loss_accumulator.keys():
            loss_accumulator[k] += loss_dict[k]

        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.detach().cpu().numpy())
        all_labels.extend(y.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')

    for k in loss_accumulator.keys():
        loss_accumulator[k] /= len(dataloader)

    return avg_loss, acc, f1, loss_accumulator


def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = F.cross_entropy(logits, y)

            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(y.detach().cpu().numpy())

    avg_loss = total_loss / len(dataloader)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return avg_loss, acc, f1, cm

# ------------------------------------------------------------------------------
# 6. Main Training Loop & Ablation Study
# ------------------------------------------------------------------------------

def get_dataloaders(train_dataset, test_dataset, batch_size, seed):
    """
    매번 동일한 순서를 보장하기 위해 Generator와 DataLoader를 새로 생성하는 함수
    """
    g = torch.Generator()
    g.manual_seed(seed)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              num_workers=2, worker_init_fn=seed_worker, generator=g)

    # Test는 shuffle=False이므로 사실 순서가 고정되지만, 일관성을 위해 같이 생성
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                             num_workers=2, worker_init_fn=seed_worker, generator=g)

    return train_loader, test_loader

def run_training(model_name, train_dataset, test_dataset, batch_size, seed, device,
                 lambda_dyn, lambda_flow, lambda_proto, lambda_contrast,
                 num_epochs=50, learning_rate=0.001, input_channels=23, num_classes=12):

    # [중요] 학습 시작 직전에 시드를 재설정하여 난수 상태 초기화
    set_seed(seed)

    # [중요] 초기화된 시드로 DataLoader를 새로 생성 -> 항상 동일한 배치 순서 보장
    train_loader, test_loader = get_dataloaders(train_dataset, test_dataset, batch_size, seed)

    print("\n" + "="*80)
    print(f" START TRAINING: {model_name}")
    print(f" Configuration -> Lambda_dyn: {lambda_dyn}, Lambda_flow: {lambda_flow}")
    print(f" Data Order Check -> First batch label sample: {next(iter(train_loader))[1][0].item()}") # 순서 확인용 로그
    print("="*80)

    # 모델 초기화
    model = ASFDCLClassifier(
        input_channels=input_channels,
        latent_dim=64,
        hidden_dim=64,
        num_classes=num_classes,
        num_heads=4,
        projection_dim=128
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_acc = 0.0
    best_f1 = 0.0
    save_path = f'best_model_{model_name}.pth'

    for epoch in range(num_epochs):
        train_loss, train_acc, train_f1, loss_dict = train_epoch(
            model, train_loader, optimizer, device,
            lambda_dyn=lambda_dyn,
            lambda_flow=lambda_flow,
            lambda_proto=lambda_proto,
            lambda_contrast=lambda_contrast,
            dyn_classes=(3, 4, 5, 6, 7, 8, 9, 10, 11),
            static_classes=(0, 1, 2)
        )

        test_loss, test_acc, test_f1, _ = evaluate(model, test_loader, device)
        scheduler.step()

        if test_f1 > best_f1:
            best_f1 = test_f1
            best_acc = test_acc
            torch.save(model.state_dict(), save_path)

        if (epoch + 1) % 10 == 0:
            print(f"[{epoch+1:02d}/{num_epochs}] "
                  f"Train Loss: {train_loss:.3f} | Train F1: {train_f1:.4f} | "
                  f"Test F1: {test_f1:.4f} (Best: {best_f1:.4f})")

    # 최종 로드 및 확인
    model.load_state_dict(torch.load(save_path))
    _, final_acc, final_f1, _ = evaluate(model, test_loader, device)

    print(f"\n>> {model_name} Final Best Result -> Acc: {final_acc:.4f}, F1: {final_f1:.4f}")
    return final_acc, final_f1


def main():
    SEED = 42
    # 초기 시드 설정 (데이터셋 Split용)
    set_seed(SEED)

    DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET'
    BATCH_SIZE = 64
    NUM_EPOCHS = 25
    LEARNING_RATE = 0.001
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    LAMBDA_PROTO = 0.05
    LAMBDA_CONTRAST = 0.2

    # 1. 데이터셋 로드 (공통)
    try:
        full_dataset = MHEALTHDataset(DATA_PATH, window_size=128, step_size=64)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return

    # 2. 데이터셋 분할 (고정) - 여기서 시드를 사용하여 분할하므로 구성 요소는 항상 동일
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(
        full_dataset, [train_size, test_size],
        generator=torch.Generator().manual_seed(SEED)
    )

    # ---------------------------------------------------------
    # Ablation Study 실행
    # 여기서는 Loader를 넘기지 않고 Dataset과 Seed를 넘김
    # ---------------------------------------------------------
    results = {}

    # Case 1: Baseline
    acc_base, f1_base = run_training(
        model_name="Baseline",
        train_dataset=train_dataset, # Dataset 전달
        test_dataset=test_dataset,
        batch_size=BATCH_SIZE,
        seed=SEED,                   # Seed 전달 -> 내부에서 재설정
        device=DEVICE,
        lambda_dyn=0.0,
        lambda_flow=0.0,
        lambda_proto=LAMBDA_PROTO,
        lambda_contrast=LAMBDA_CONTRAST,
        num_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE
    )
    results['Baseline'] = {'Acc': acc_base, 'F1': f1_base}

    # Case 2: Full Model
    acc_full, f1_full = run_training(
        model_name="Full_Model",
        train_dataset=train_dataset, # 동일한 Dataset 전달
        test_dataset=test_dataset,
        batch_size=BATCH_SIZE,
        seed=SEED,                   # 동일한 Seed 전달 -> 동일한 순서 보장
        device=DEVICE,
        lambda_dyn=0.05,
        lambda_flow=0.02,
        lambda_proto=LAMBDA_PROTO,
        lambda_contrast=LAMBDA_CONTRAST,
        num_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE
    )
    results['Full_Model'] = {'Acc': acc_full, 'F1': f1_full}

    # ---------------------------------------------------------
    # 결과 비교
    # ---------------------------------------------------------
    print("\n" + "="*60)
    print(" ABLATION STUDY RESULTS COMPARISON")
    print("="*60)
    print(f"{'Model Setting':<20} | {'Test Accuracy':<15} | {'Test F1-Score':<15}")
    print("-" * 60)

    print(f"{'Baseline (0.0/0.0)':<20} | {results['Baseline']['Acc']:.4f}{' '*10} | {results['Baseline']['F1']:.4f}")
    print(f"{'Full Model':<20} | {results['Full_Model']['Acc']:.4f}{' '*10} | {results['Full_Model']['F1']:.4f}")
    print("-" * 60)

    diff_f1 = results['Full_Model']['F1'] - results['Baseline']['F1']
    print(f"Improvement (F1): {diff_f1:+.4f}")
    print("="*60)

if __name__ == "__main__":
    main()

Found 10 log files in /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET
Loaded MHEALTH dataset
  X shape : (5361, 128, 23)  (N, T, C)
  y shape : (5361,)  (N,)
  Classes : 12

 START TRAINING: Baseline
 Configuration -> Lambda_dyn: 0.0, Lambda_flow: 0.0
 Data Order Check -> First batch label sample: 7
[10/25] Train Loss: 0.593 | Train F1: 0.9432 | Test F1: 0.9646 (Best: 0.9646)
[20/25] Train Loss: 0.433 | Train F1: 0.9905 | Test F1: 0.9765 (Best: 0.9786)

>> Baseline Final Best Result -> Acc: 0.9804, F1: 0.9786

 START TRAINING: Full_Model
 Configuration -> Lambda_dyn: 0.05, Lambda_flow: 0.02
 Data Order Check -> First batch label sample: 7
[10/25] Train Loss: 0.464 | Train F1: 0.9800 | Test F1: 0.9739 (Best: 0.9767)
[20/25] Train Loss: 0.378 | Train F1: 0.9985 | Test F1: 0.9848 (Best: 0.9863)

>> Full_Model Final Best Result -> Acc: 0.9897, F1: 0.9875

 ABLATION STUDY RESULTS COMPARISON
Model Setting        | Test Accuracy   | Test F1-Score  
-----------------------------